<a href="https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/Cutting_Stock_Problem_Compact_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install coinor-cbc
#!apt-get install glpk-utils
!pip install pyomo

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
coinor-cbc is already the newest version (2.10.7+ds1-1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [2]:
import pyomo.environ as pyo

## Data

In [3]:
L = 120   #Stock length (cm)
m = 40    #An alternative worst upper bound is sum(demand)

# item types: lengths (cm) and demands (units)
item_lengths = [19, 36, 62, 25, 48]
demand = [10, 12, 15, 31, 17]
n = len(demand)

## Model

In [4]:
# -------- Model --------
model = pyo.ConcreteModel()

# Index Sets
model.I = range(m)
model.J = range(n)

# Parameters
model.L = L
model.l = item_lengths
model.d = demand

# Decision variables
# x[i,j] = number of pieces of type j cut from stock piece i (non-negative integer)
model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeIntegers)

# y[i] = 1 if stock piece i is used (cut), 0 otherwise
model.y = pyo.Var(model.I, domain=pyo.Binary)

# Knapsack Constraints
def capacity_rule(model, i):
    # sum_j l_j * x_ij <= L * y_i
    return sum(model.l[j] * model.x[i, j] for j in model.J) <= model.L * model.y[i]
model.knapsack = pyo.Constraint(model.I, rule=capacity_rule)

def demand_rule(model, j):
    # sum_i x_ij >= d_j
    return sum(model.x[i, j] for i in model.I) >= model.d[j]

model.demand = pyo.Constraint(model.J, rule=demand_rule)

# Objective: minimize number of stock pieces used
def obj_rule(model):
    return sum(model.y[i] for i in model.I)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)  # default sense=1 (minimize)

In [5]:
solver = pyo.SolverFactory("cbc")
solver.options["Sec"] = 600 ##Limits execution time in 600 seconds maximum
result = solver.solve(model, tee=True)  # tee=True prints solver output

Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -Sec 600 -printingOptions all -import /tmp/tmp9gj51ek2.pyomo.lp -stat=1 -solve -solu /tmp/tmp9gj51ek2.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 600
Option for printingOptions changed from normal to all
Presolve 45 (0) rows, 240 (0) columns and 440 (0) elements
Statistics for presolved model
Original problem has 240 integers (40 of which binary)
==== 200 zero objective 2 different
200 variables have objective of 0
40 variables have objective of 1
==== absolute objective values 2 different
200 variables have objective of 0
40 variables have objective of 1
==== for integers 200 zero objective 2 different
200 variables have objective of 0
40 variables have objective of 1
==== for integers absolute objective values 2 different
200 variables have objective of 0
40 variables have objective of 1
===== end objective counts


Problem has 45 rows, 240 columns (40 with ob

In [6]:
print("\n--- Solution summary ---")
print("Objective (number of stocks used):", pyo.value(model.obj))

# Show which stocks are used and their cuts
used_stocks = [i for i in model.I if pyo.value(model.y[i]) >= 0.5]
print("Number of stocks used (y_i = 1):", len(used_stocks))
print("Used stock indices:", used_stocks)

for i in used_stocks:
        cuts = {j: int(round(pyo.value(model.x[i, j]))) for j in model.J if pyo.value(model.x[i, j]) > 1e-6}
        total_width = sum(item_lengths[j] * cuts[j] for j in cuts)
        print(f" Stock {i}: cuts {cuts}  total used width = {total_width} / {L}")

# Aggregate by item type (check demand fulfillment)
print("\nTotal pieces produced by type:")
for j in model.J:
    produced = sum(pyo.value(model.x[i, j]) for i in model.I)
    print(f" Item {j} (length {item_lengths[j]}): produced = {int(round(produced))}  demand = {demand[j]}")


--- Solution summary ---
Objective (number of stocks used): 28.0
Number of stocks used (y_i = 1): 28
Used stock indices: [0, 1, 3, 4, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 24, 25, 27, 28, 30, 31, 32, 33, 34, 35, 38, 39]
 Stock 0: cuts {0: 1, 3: 4}  total used width = 119 / 120
 Stock 1: cuts {2: 1, 3: 2}  total used width = 112 / 120
 Stock 3: cuts {4: 2}  total used width = 96 / 120
 Stock 4: cuts {1: 2, 4: 1}  total used width = 120 / 120
 Stock 9: cuts {0: 1, 4: 2}  total used width = 115 / 120
 Stock 10: cuts {1: 2, 4: 1}  total used width = 120 / 120
 Stock 11: cuts {1: 2, 4: 1}  total used width = 120 / 120
 Stock 12: cuts {0: 1, 1: 1, 2: 1}  total used width = 117 / 120
 Stock 13: cuts {2: 1, 4: 1}  total used width = 110 / 120
 Stock 14: cuts {2: 1, 4: 1}  total used width = 110 / 120
 Stock 15: cuts {0: 1, 4: 2}  total used width = 115 / 120
 Stock 16: cuts {2: 1, 3: 2}  total used width = 112 / 120
 Stock 18: cuts {2: 1, 3: 2}  total used width = 112 / 120
 Stock 19